In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import plotly.express as px
# text API's
import re
import spacy
import unicodedata
from nltk import TweetTokenizer
from spacy.lang.es import Spanish
from spacy.lang.en import English
import nltk
from nltk.corpus import stopwords
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.preprocessing import LabelEncoder 
from sklearn import preprocessing
from sklearn.metrics import plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

In [9]:
df = pd.read_csv('./data.csv', sep=',')
df.head(10)

,id,date,energyreal_wac_sum_consumed,energyreal_wac_sum_produced,current_ac_sum,powerreal_p_sum,reference,voltage_ac_phase_average
0,3108530,"['May', '16', '2022', '00:00:00']",33843600.0,242388.0,65.15810,7403.190,comfenalco1-meter,127.000
1,3108531,"['May', '16', '2022', '00:00:00']",3202100.0,12528600.0,5.14009,342.882,takurika1-meter,125.047
2,3108532,"['May', '16', '2022', '00:00:00']",16537400.0,22086.0,44.60990,5397.920,edlv-meter,126.678
3,3108533,"['May', '16', '2022', '00:00:00']",33843700.0,242388.0,64.47030,7308.290,comfenalco1-meter,126.997
4,3108534,"['May', '16', '2022', '00:00:00']",3202100.0,12528600.0,5.14234,343.748,takurika1-meter,125.156
5,3108535,"['May', '16', '2022', '00:00:00']",33843700.0,242388.0,64.66300,7339.680,comfenalco1-meter,127.154
6,3108536,"['May', '16', '2022', '00:00:00']",3202100.0,12528600.0,5.13959,343.197,takurika1-meter,125.295
7,3108537,"['May', '16', '2022', '00:00:00']",33843700.0,242388.0,64.23390,7270.400,comfenalco1-meter,127.009
8,3108538,"['May', '16', '2022', '00:00:00']",3202100.0,12528600.0,5.13756,343.246,takurika1-meter,125.128
9,3108539,"['May', '16', '2022', '00:00:00']",33843700.0,242388.0,64.12690,7258.180,comfenalco1-meter,127.209


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1170 entries, 0 to 1169
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           1170 non-null   int64  
 1   date                         1170 non-null   object 
 2   energyreal_wac_sum_consumed  1170 non-null   float64
 3   energyreal_wac_sum_produced  1170 non-null   float64
 4   current_ac_sum               1170 non-null   float64
 5   powerreal_p_sum              1170 non-null   float64
 6   reference                    1170 non-null   object 
 7   voltage_ac_phase_average     1170 non-null   float64
dtypes: float64(5), int64(1), object(2)
memory usage: 73.2+ KB


In [11]:
df.isnull().sum()

id                             0
date                           0
energyreal_wac_sum_consumed    0
energyreal_wac_sum_produced    0
current_ac_sum                 0
powerreal_p_sum                0
reference                      0
voltage_ac_phase_average       0
dtype: int64

In [13]:
data_raw['reference'].value_counts()

comfenalco1-meter    546
takurika1-meter      512
edlv-meter           112
Name: reference, dtype: int64

In [14]:
data_raw['voltage_ac_phase_average'].value_counts()

130.025    4
126.859    3
123.498    3
123.404    3
129.710    3
          ..
124.085    1
127.205    1
127.293    1
124.202    1
129.399    1
Name: voltage_ac_phase_average, Length: 1055, dtype: int64

In [15]:
data_raw['powerreal_p_sum'].value_counts()

 329.341     3
 4064.290    2
 329.667     2
 3755.750    2
 330.162     2
            ..
 328.960     1
 3916.740    1
 3969.390    1
 329.274     1
-6606.480    1
Name: powerreal_p_sum, Length: 1138, dtype: int64

In [16]:
data_raw['current_ac_sum'].value_counts()

 4.83064     3
 38.47280    3
 39.12050    2
 22.42500    2
 4.80441     2
            ..
 4.78166     1
 4.77529     1
 5.16256     1
 37.63470    1
-56.06040    1
Name: current_ac_sum, Length: 1147, dtype: int64

In [17]:
data_raw['energyreal_wac_sum_produced'].value_counts()

235658.0    10
235537.0    10
242245.0     9
241909.0     8
242056.0     8
            ..
10200.0      1
17846.0      1
10532.0      1
17738.0      1
15038.0      1
Name: energyreal_wac_sum_produced, Length: 319, dtype: int64

In [18]:
data_raw['energyreal_wac_sum_consumed'].value_counts()

24309700.0    10
24382600.0    10
2138860.0      8
32538200.0     8
2125660.0      8
              ..
12277400.0     1
12347800.0     1
20864000.0     1
8621680.0      1
33843600.0     1
Name: energyreal_wac_sum_consumed, Length: 421, dtype: int64